In [2]:
import os
import pandas
import time
import csv
import json
from ipywidgets import interact, IntSlider
from IPython.display import display

def freeze_header(df, num_rows=30, num_columns=10, step_rows=1,
                  step_columns=1):

    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])
with open("imperator/mappings/countries_names.csv") as countries_names:
    countries_names = pandas.read_csv(countries_names, sep=";", header=None)
    countries_names.columns = ["country_name_key", "country_name"]
filename = "mp_677"
savegame = "../Imperator/save games/"+filename+".json"
with open(savegame, encoding="utf-8") as source:
#with open("../Imperator/workbench/4.json", encoding="utf-8") as source:
    source=json.load(source)
    


In [3]:
source["date"]
#country_ID='source["country"]["country_database"][x]'
country_attributes={
    
    "TAG":'["tag"]',
    "graphical_culture":'["graphical_culture"]',
    "manpower":'["currency_data"]["manpower"]',
    "gold":'["currency_data"]["gold"]',
    "stability":'["currency_data"]["stability"]',
    "tyranny":'["currency_data"]["tyranny"]',
    "war_exhaustion":'["currency_data"]["war_exhaustion"]',
    "aggressive_expansion":'["currency_data"]["aggressive_expansion"]',
    "political_influence":'["currency_data"]["political_influence"]',
    "military_experience":'["currency_data"]["military_experience"]',
    "tax_income":'["economy"]["lastyearincome"][:2][0]',
    "trade_income":'["economy"]["lastyearincome"][:2][1]',
    "capital":'["capital"]',
    "diplomatic_stance":'["diplomatic_stance"]',
    "military_tech":'["technology"]["military_tech"]["level"]',
    "civic_tech":'["technology"]["civic_tech"]["level"]',
    "oratory_tech":'["technology"]["oratory_tech"]["level"]',
    "religious_tech":'["technology"]["religious_tech"]["level"]',
    "military_progress":'["technology"]["military_tech"]["progress"]',
    "civic_progress":'["technology"]["civic_tech"]["progress"]',
    "oratory_progress":'["technology"]["oratory_tech"]["progress"]',
    "religious_progress":'["technology"]["religious_tech"]["progress"]',
    "total_holdings":'["total_holdings"]',
    "possible_holdings":'["possible_holdings"]',
    "total_power_base":'["total_power_base"]',
    "non_loyal_power_base":'["non_loyal_power_base"]',
    "total_cohorts":'["total_cohorts"]',
    "centralization":'["centralization"]',
    "total_population":'["total_population"]',
    "country_name_key":'["country_name"]["name"]',
    "max_manpower":'["max_manpower"]',
    #"government_key":'["government_key"]',
    #"primary_culture":'["primary_culture"]',
    #"religion":'["religion"]',
    #"heritage":'["heritage"]',
    #"country_type":'["country_type"]',
}
countryattributeslist =""
for x in country_attributes.keys():
    countryattributeslist += 'source["country"]["country_database"][x]'+ str(country_attributes[x]) + str(",")

countryattributeslist = countryattributeslist[:len(countryattributeslist)-1]

country_values = [eval(countryattributeslist) for x in source["country"]["country_database"].keys()]
# filter:  if source["country"]["country_database"][x]["currency_data"]["stability"]>0)
countries_df = pandas.DataFrame(country_values,index=(x for x in source["country"]["country_database"].keys()))
countries_df.columns = country_attributes.keys()

interest_list=["ROM","SEL","CAR","EGY","MRY", "SCY"]
#interest_list=["BAR"]

#countries_df.assign(miltech=lambda x: x.military_tech + x.military_progress/100)
countries_df["military_tech"]=round(countries_df["military_tech"]+countries_df["military_progress"]/100,2)
countries_df["civic_tech"]=round(countries_df["civic_tech"]+countries_df["civic_progress"]/100,2)
countries_df["oratory_tech"]=round(countries_df["oratory_tech"]+countries_df["oratory_progress"]/100,2)
countries_df["religious_tech"]=round(countries_df["religious_tech"]+countries_df["religious_progress"]/100,2)
countries_df["total_income"]=round(countries_df["tax_income"]+countries_df["trade_income"],2)
date = int(source["date"][:source["date"].find(".")])-753
countries_df["Année"]= date
real_mp = countries_df["manpower"]*500
countries_df["real_manpower"]= round(real_mp,2)
countries_df["max_manpower"]=countries_df["max_manpower"]*500

#countries_df.merge(source["country"]["country_database"][countries_df.keys]["government_key"], how='Left',on=None)

countries_df = countries_df.merge(countries_names, how='left', on='country_name_key')
countries_df = countries_df.drop(columns=["religious_progress","oratory_progress","civic_progress","military_progress"])

#countries_df = countries_df[countries_df["TAG"].isin(interest_list)]

#countries_df = countries_df.merge(countries_names, how='left', on=["capital","country_name_key"])

cols_at_start = ["total_income",'TAG','country_name','Année',"max_manpower","real_manpower"]
countries_df = countries_df[[c for c in countries_df if c in cols_at_start]+[c for c in countries_df if c not in cols_at_start]]
countries_df = countries_df.sort_values("total_holdings",ascending=False).query('total_income > 100 & total_holdings > 0')

freeze_header(countries_df)
#countries_df.plot.scatter(x='trade_income',y='tax_income')
#countries_df


interactive(children=(IntSlider(value=30, description='rows', max=42, min=30, readout=False), IntSlider(value=…